In [29]:
%load_ext autoreload
%autoreload 2
import sys
import os
current_dir = os.getcwd()
libs_path = os.path.abspath(os.path.join(current_dir, "..", "libs"))
if libs_path not in sys.path:
    sys.path.append(libs_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
import pandas as pd
from IPython.display import display
from ipywidgets import interact
import numpy as np
pd.set_option('display.precision', 3)
import data_processing as dp

In [31]:
res=dp.load_datasets()

SelectMultiple(description='Датасеты:', layout=Layout(width='500px'), options={'.ipynb_checkpoints': 'E:\\stud…

Button(description='ОК', style=ButtonStyle())

Output()

In [32]:
datasets = []

for fn in res.options:

    if res.options[fn] not in res.value:
        continue
    dataset = dp.Dataset(pd.read_csv(res.options[fn]), name=fn)

    dataset.target_features = ["Cover_Type"]
    datasets.append(dataset)

In [37]:
for d in datasets:
    print(d.name)

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer


binary_features_all = ["Wilderness_Area", "Soil_Type"]

df_enhanced.csv
df_enhanced.csv_selected_features.csv
df_enhanced_one-hot.csv
df_enhanced_one-hot.csv_selected_features.csv
V4_classification_lr3.csv


In [40]:

for d in datasets:
    print(d.name)

    if "one-hot" not in d.name:
        soil_type_features = []
        for feature in d.columns:
            if feature[:5] == "Soil_Type_0"[:5]:
                soil_type_features.append(feature)

        wilderness_area_features = []
        for feature in d.columns:
            if feature[:5] == "Wilderness_Area_0"[:5]:
                wilderness_area_features.append(feature)
        binary_features_all = wilderness_area_features + soil_type_features

    else:
        binary_features_all = ["Wilderness_Area", "Soil_Type"]
        
    # Создаем препроцессор для масштабирования числовых признаков
    preprocessor_original = ColumnTransformer(
        transformers=[
            ('num', MinMaxScaler(), list((set(d.columns) - set(binary_features_all) - set(d.target_features))) )],
        remainder='passthrough' # Оставляем остальные признаки как есть (бинарные)
    )

    XXX = preprocessor_original.fit_transform(d.drop(d.target_features, axis=1))
    df_scaled = pd.DataFrame(XXX, columns=(list((set(d.columns) - set(binary_features_all) - set(d.target_features))) + binary_features_all))

    df_scaled[d.target_features] = d[d.target_features].values # Добавляем целевой признак

    df_scaled.to_csv(f"../datasets/{d.name}_scaled.csv", index=False)


df_enhanced.csv
df_enhanced.csv_selected_features.csv
df_enhanced_one-hot.csv
df_enhanced_one-hot.csv_selected_features.csv
V4_classification_lr3.csv
